# Software Dependencies

- pandas: `pip install pandas`
- numpy: `pip install numpy`
- matplotlib: `pip install matplotlib`
- graph-tool: `conda install -c conda-forge graph-tool` ([doc](https://git.skewed.de/count0/graph-tool/-/wikis/installation-instructions))

# Data Descriptions

1) `matches.csv`

- `MatchID`: A unqiue identifier for each match played during the season, and reflects the order of the match in the season.
- `OpponentID`: A unqiue identifier for the opposing team played in the match. **Note that the Huskies play each opposing team twice during the season**
- `Outcome`: Result of the match, eiter `win`, `loss`, or `tie`.
- `OwnScore`: Number of goals scored by the Huskies.
- `OpponentScore`: Number of goals scored by the Opposing Team.
- `Side`: Whether the Huskies were the `home` team or `away` team.
- `CoachID`: A unqiue identifier for the Huskies coach for this match.

2) `passingevents.csv`

- `MatchID`: A unqiue identifier for each match played during the season (see `matches.csv`).
- `TeamID`: A unqiue identifier for the team involved in the pass (either `Huskies` or `OpponentID` from `matches.csv`).
- `OriginPlayerID`: A unqiue identifier for the Player at the origin of the pass. The `PlayerID` has the form `TeamID_PlayerPosition##` where `TeamID` denotes the team on which the player plays and `PlayerPosition` reflects the player's position. Possible positions are: `F`:forward, `D`:defense, `M`:midfield, or `G`:goalkeeper
- `DestinationPlayerID`: A unqiue identifier for the Player at the destination of the pass. (see `OriginPlayerID`)
- `MatchPeriod`: The half in which the event took place. `1H`: first half, `2H`: second half
- `EventTime`: The time in seconds during the `MatchPeriod` (1st or 2nd half) at which the event took place.
- `EventSubType`: The type of pass made. Can be one of: `Head pass`, `Simple pass`, `Launch`, `High pass`, `Hand pass`, `Smart pass`, `Cross`.
- `EventOrigin_x`: The x-coordinate on the field at which the pass originated. The x-coordinate is in the range $[0, 100]$ and is **oriented from the perspective of the attacking team**, where $0$ indicates the team's own goal, and $100$ indicates the oppositing team's goal.
- `EventOrigin_y`: The y-coordinate on the field at which the pass originated. The y-coordinate is in the range $[0, 100]$ and is **oriented from the perspective of the attacking team**, where $0$ indicates the team's left-hand side, and $100$ indicates the team's right-hand side.
- `EventDestination_x`: The x-coordinate on the field at the pass destination. (see `EventOrigin_x`)
- `EventDestination_y`: The y-coordinate on the field at the pass destination.  (see `EventOrigin_y`)

3) `fullevents.csv`

- `MatchID`: A unqiue identifier for each match played during the season (see `matches.csv`).
- `TeamID`: A unqiue identifier for the team involved in the pass (either `Huskies` or `OpponentID` from `matches.csv`).
- `OriginPlayerID`: A unqiue identifier for the Player initiating the event.  The `PlayerID` has the form `TeamID_PlayerPosition##` where `TeamID` denotes the team on which the player plays and `PlayerPosition` reflects the player's position.  Possible positions are: `F`:forward, `D`:defense, `M`:midfield, or `G`:goalkeeper
- `DestinationPlayerID`: A unqiue identifier for the Player at the destination of the event. (see `OriginPlayerID`) **NOTE: Only valid for `Pass` or `Subsition` event types, otherwise `NaN`.**
- `MatchPeriod`: The half in which the event took place.  `1H`: first half, `2H`: second half
- `EventTime`: The time in seconds during the `MatchPeriod` (1st or 2nd half) at which the event took place.
- `EventType`: The type of the event. Can be one of: `Free Kick`, `Duel`, `Pass`, `Others on the ball`, `Foul`, `Goalkeeper leaving line`, `Offside`, `Save attempt`, `Shot`, `Substitution`, `Interruption`
- `EventSubType`: The subtype of the event. Can be one of: `Goal kick`, `Air duel`, `Throw in`, `Head pass`, `Ground loose ball duel`, `Simple pass`, `Launch`, `High pass`, `Touch`, `Ground defending duel`, `Hand pass`, `Ground attacking duel`, `Foul`, `Free kick cross`, `Goalkeeper leaving line`, ` `, `Free Kick`, `Smart pass`, `Cross`, `Save attempt`, `Corner`, `Clearance`, `Shot`, `Acceleration`, `Reflexes`, `Substitution`, `Late card foul`, `Simulation`, `Free kick shot`, `Protest`, `Hand foul`, `Penalty`, `Violent Foul`, `Whistle`, `Out of game foul`, `Ball out of the field`, `Time lost foul`
- `EventOrigin_x`: The x-coordinate on the field at which the event originated. The x-coordinate is in the range $[0, 100]$ and is **oriented from the perspective of the attacking team**, where $0$ indicates the team's own goal, and $100$ indicates the oppositing team's goal.
- `EventOrigin_y`: The y-coordinate on the field at which the event originated. The y-coordinate is in the range $[0, 100]$ and is **oriented from the perspective of the attacking team**, where $0$ indicates the team's left-hand side, and $100$ indicates the team's right-hand side.
- `EventDestination_x`: The x-coordinate on the field at the event destination. (see `EventOrigin_x`)
- `EventDestination_y`: The y-coordinate on the field at the event destination. (see `EventOrigin_y`) 

**NOTE: For `Substitution` events, the Outgoing player is the `OriginPlayerID`, while in the incoming player is the `DestinationPlayerID`**

## Details for `EventType`

- Subtype of Duel: `['Air duel', 'Ground attacking duel', 'Ground defending duel', 'Ground loose ball duel']`
- Subtype of Foul: `['Foul', 'Hand foul', 'Late card foul', 'Out of game foul', 'Protest', 'Simulation', 'Time lost foul', 'Violent Foul']`
- Subtype of Free Kick: `['Corner', 'Free Kick', 'Free kick cross', 'Free kick shot', 'Goal kick', 'Penalty', 'Throw in']`
- Subtype of Goalkeeper leaving line: `['Goalkeeper leaving line']`
- Subtype of Interruption: `['Ball out of the field', 'Whistle']`
- Subtype of Offside: `[nan]`
- Subtype of Others on the ball: `['Acceleration', 'Clearance', 'Touch']`
- Subtype of Pass: `['Cross', 'Hand pass', 'Head pass', 'High pass', 'Launch', 'Simple pass', 'Smart pass']`
- Subtype of Save attempt: `['Reflexes', 'Save attempt']`
- Subtype of Shot: `['Shot']`
- Subtype of Substitution: `['Substitution']`

# Attack Index

# Objectives

- Create a network for the ball passing between players, where each player is a node and each pass constitutes a link between players.
    
    Use your passing network to identify network patterns, such as **dyadic** and **triadic** configurations and team formations. Also consider other structural indicators and network properties across the games. You should explore multiple scales such as, but not limited to, micro (pairwise) to macro (all players) when looking at interactions, and time such as short (minute-to-minute) to long (entire game or entire season).


- Identify performance indicators that reflect successful teamwork (in addition to points or wins) such as diversity in the types of plays, coordination among players or distribution of contributions.

    You also may consider other team level processes, such as adaptability, flexibility, tempo,or flow. It may be important to clarify whether strategies are universally effective or dependent on opponents’ counter-strategies. Use the performance indicators and team level processes that you have identified to create a model that captures structural, configurational, and dynamical aspects of teamwork.


- Use the insights gained from your teamwork model to inform the coach about what kinds of structural strategies have been effective for the Huskies.

    Advise the coach on what changes the network analysis indicates that they should make next season to improve team success.


- Your analysis of the Huskies has allowed you to consider group dynamics in a controlled setting of a team sport. Understanding the complex set of factors that make some groups perform better than others is critical for how societies develop and innovate. As our societies increasingly solve problems involving teams, can you generalize your findings to say something about how to design more effective teams? What other aspects of teamwork would need to be captured to develop generalized models of team performance?

Your submission should consist of:

- One-page Summary Sheet
- Table of Contents
- Your solution of no more than 20 pages, for a maximum of 22 pages with your summary and table of contents.

**Note**: Reference List and any appendices do not count toward the page limit and should appear after your completed solution. You should not make use of unauthorized images and materials whose use is restricted by copyright laws. Ensure you cite the sources for your ideas and the materials used in your report.

# Initialization

In [1]:
data_matches_path = 'data/matches.csv'
data_passings_path = 'data/passingevents.csv'
data_events_path = 'data/fullevents.csv'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 120

In [3]:
matches_df = pd.read_csv(data_matches_path)
passings_df = pd.read_csv(data_passings_path)
events_df = pd.read_csv(data_events_path)


# convert 'win', 'tie' and 'lose' to int
def outcome_int_map(x: str):
    if x == 'win':
        return 1
    elif x == 'tie':
        return 0
    else:
        return -1


matches_df['Outcome'] = matches_df['Outcome'].apply(outcome_int_map)

all_events = events_df.join(
    matches_df[['OwnScore', 'OpponentScore', 'Outcome']],
    on='MatchID',
    how='outer')
huskies_passes = all_events[(all_events['TeamID'] == 'Huskies')
                            & (all_events['EventType'] == 'Pass')]
huskies_events = all_events[all_events['TeamID'] == 'Huskies']

In [4]:
huskies_player_ids = np.unique(
    huskies_events['OriginPlayerID'].tolist() +
    huskies_events['DestinationPlayerID'].tolist()).tolist()
huskies_player_ids.remove('nan')

match_ids = np.unique(matches_df['MatchID'])

# Compute metrics for a network

*Defining a historic football team: Using NetworkScience to analyze Guardiola’s F.C. Barcelona*

1. Number of passes

In [8]:
def get_n_passes(match_id: int):
    return passings_df[(passings_df['MatchID'] == match_id)
                       & (passings_df['TeamID'] == 'Huskies')].shape[0]

2. Number of shots to goal

In [9]:
def get_n_shots(match_id: int):
    return events_df[(events_df['EventType'] == 'Shot')
                     & (events_df['MatchID'] == match_id) &
                     (events_df['TeamID'] == 'Huskies')].shape[0]

3. Number of goals

In [10]:
def get_score(match_id: int):
    return matches_df[matches_df['MatchID'] ==
                      match_id]['OwnScore'].tolist()[0]


get_n_goals = get_score

4. Number of points at the end of season

NONE

$X$ and $Y$ centroid coordinates correspond to the average position of all pases of the network. Specifically, we **only consider the position from where the pass is sent**

5. X-coordinate of the network centroid $X$
6. Y-coordinate of the network centroid $Y$

In [11]:
from typing import Tuple


def get_centroids(match_id: int) -> Tuple[float, float]:
    data = passings_df[(passings_df['MatchID'] == match_id)
                       & (passings_df['TeamID'] == 'Huskies')]
    return data['EventOrigin_x'].mean(), data['EventOrigin_y'].mean()


def get_x_centroid(match_id: int):
    return get_centroids(match_id)[0]


def get_y_centroid(match_id: int):
    return get_centroids(match_id)[1]

7. Dispersion of the position of the players around the network centroid $NC_\text{disp}$

The centroid dispersion $C\text{ent}_\text{disp}$ corresponds to the standard deviation of the distances of the players with regard to the position of the network centroid.

**FIXME: how to calculate the position of a player? Assuming the coordinates when initiating a pass**

In [12]:
def get_dispersion(match_id: int):
    data = passings_df[(passings_df['MatchID'] == match_id)
                       & (passings_df['TeamID'] == 'Huskies')]
    cx = data['EventOrigin_x'].mean()
    cy = data['EventOrigin_y'].mean()
    dists = [(r['EventOrigin_x'] - cx)**2 + (r['EventOrigin_y'] - cy)**2
             for i, r in data.iterrows()]
    dists = np.sqrt(np.asarray(dists))
    return np.std(dists)

8. Average ratio between the passing distance parallel and perpendicular to the opponent’s goal $\Delta y/\Delta x$

In [13]:
def get_dx_div_dy(match_id: int):
    data = passings_df[(passings_df['MatchID'] == match_id)
                       & (passings_df['TeamID'] == 'Huskies')]
    ratios = [
        (r['EventDestination_y'] - r['EventOrigin_y']) /
        (r['EventDestination_x'] - r['EventOrigin_x'] + 10**-5
         )  # prevent divided by zero
        for i, r in data.iterrows()
    ]
    return np.mean(np.asarray(ratios))

## My metrics

In [14]:
def get_n_attempt_shots(match_id: int):
    return get_n_shots(match_id) - get_n_goals(match_id)

## Evaluation methods

In [15]:
def get_score_diff(match_id: int):
    data = matches_df[matches_df['MatchID'] == match_id]
    return data['OwnScore'].tolist()[0] - data['OpponentScore'].tolist()[0]


def get_outcome(match_id: int):
    return matches_df[matches_df['MatchID'] == match_id]['Outcome'].tolist()[0]

# Plot metrics against evaluation

In [16]:
from typing import List


def plot_metrics_against(y_func, y_name: str):
    metric_funcs = [
        get_n_passes, get_n_shots, get_n_goals, get_x_centroid, get_y_centroid,
        get_dispersion, get_dx_div_dy, get_n_attempt_shots
    ]
    metric_names = [
        'Number of passes', 'Number of shots', 'Number of goals',
        r'$X$ centroid', r'$Y$ centroid', 'Dispersion',
        r'$\frac{\Delta y}{\Delta x}$', 'Attempted Shots'
    ]
    n = len(metric_names)
    fig, axs = plt.subplots(4, 2)
    fig.set_size_inches(18, 15)
    Y = [y_func(mi) for mi in match_ids]
    for i in range(n):
        X = [metric_funcs[i](mi) for mi in match_ids]
        ax_x = i // 2
        ax_y = i % 2
        axs[ax_x, ax_y].scatter(X, Y)
        axs[ax_x, ax_y].set_xlabel(metric_names[i])
        axs[ax_x, ax_y].set_ylabel(y_name)
    fig.tight_layout()
    fig.suptitle("Metrics vs. {0} in each match".format(y_name), size=16)
    fig.subplots_adjust(top=0.95)
    plt.show()

In [17]:
# plot_metrics_against(get_score_diff, 'Score differences')

In [18]:
# plot_metrics_against(get_outcome, 'Score differences')

In [19]:
from typing import List


def get_metrics_eval_corr(y_funcs, y_names: List[str]):
    metric_funcs = [
        get_n_passes, get_n_shots, get_n_goals, get_x_centroid, get_y_centroid,
        get_dispersion, get_dx_div_dy, get_n_attempt_shots
    ]
    metric_names = [
        'Number of passes', 'Number of shots', 'Number of goals', 'X centroid',
        'Y centroid', 'Dispersion', 'dy/dx', 'Attempted Shots'
    ]
    df_dict = {}
    for yf, yn in zip(y_funcs, y_names):
        df_dict[yn] = [yf(mi) for mi in match_ids]

    for xf, xn in zip(metric_funcs, metric_names):
        df_dict[xn] = [xf(mi) for mi in match_ids]
    df = pd.DataFrame(df_dict)
    return df.corr()

In [20]:
# get_metrics_eval_corr([get_score_diff, get_outcome],
#                       ['Score differences', 'Outcomes'])

# Metrics from paper

*The harsh rule of the goals: Data-driven performance indicators for football teams*

1. $\mu$: the average amount of passes managed by players (in and out) in the team during the game
2. $\sigma$: the variance  of the amount of passes managed by players in the team during the game. 

In [21]:
def get_player_passes_metrics():
    """Calculate and return:
    - the total number of passes,
    - the average number of passes for players,
    - the std of number of passes for players,
    for every match.
    """
    total_player_counts = [[
        huskies_passes[(huskies_passes['MatchID'] == mi) & (
            (huskies_passes['OriginPlayerID'] == hpi)
            | (huskies_passes['DestinationPlayerID'] == hpi))].shape[0]
        for hpi in huskies_player_ids
    ] for mi in match_ids]
    passes = np.asarray(total_player_counts)
    return np.sum(passes, axis=1), np.mean(passes, axis=1), np.std(passes,
                                                                   axis=1)

In [22]:
w, mu, sigma = get_player_passes_metrics()
h = 1 / w + 1 / mu + 1 / sigma
metrics_df = pd.DataFrame({'w': w, 'mu': mu, 'sigma': sigma, 'h': h})
metrics_df = metrics_df.join(matches_df[['Outcome', 'OwnScore']])
metrics_df.corr()

,w,mu,sigma,h,Outcome,OwnScore
w,1.000000,1.000000,0.988331,-0.920100,0.131284,-0.072945
mu,1.000000,1.000000,0.988331,-0.920100,0.131284,-0.072945
sigma,0.988331,0.988331,1.000000,-0.909304,0.183129,-0.048731
h,-0.920100,-0.920100,-0.909304,1.000000,-0.143148,0.035912
Outcome,0.131284,0.131284,0.183129,-0.143148,1.000000,0.676123
OwnScore,-0.072945,-0.072945,-0.048731,0.035912,0.676123,1.000000
